# **Executable TSG for WideWorldImporters**

This notebook is adapted from Glenn Berry's [**Azure SQL Database Diagnostic Information Queries.** ](https://glennsqlperformance.com/2021/02/02/sql-server-diagnostic-information-queries-for-february-2021/).

- Originally written by Glenn Berry
- Version: August 2, 2020
- Twitter: GlennAlanBerry
- Blog: [https://glennsqlperformance.com/](https://glennsqlperformance.com/)
- YouTube: [https://bit.ly/2PkoAM1](https://bit.ly/2PkoAM1)

> **Copyright (C) 2020 Glenn Berry**  
> All rights reserved.  
> You may alter this code for your own _non-commercial_ purposes.  
> You may republish altered code as long as you include this copyright and give due credit.  
>   
> THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR A PARTICULAR PURPOSE.

# Server-Level Queries

Make sure you are connected a user database, rather than the master system database

## SQL and OS Version information for current instance  (Query 1) (Version Info)

In [32]:
-- SQL and OS Version information for current instance  (Query 1) (Version Info)
SELECT @@SERVERNAME AS [Server Name], @@VERSION AS [SQL Server and OS Version Info];

(1 row affected)

Total execution time: 00:00:00.116

Server Name,SQL Server and OS Version Info
jukoesmasqldb,Microsoft SQL Azure (RTM) - 12.0.2000.8 Feb 20 2021 17:51:58 Copyright (C) 2019 Microsoft Corporation


Useful links related to the above query:
* [Download and install Azure Data Studio](https://bit.ly/2vgke1A)

## Detect blocking (run multiple times)  (Query 9) (Detect Blocking)

In [33]:
-- Detect blocking (run multiple times)  (Query 9) (Detect Blocking)
SELECT t1.resource_type AS [lock type], DB_NAME(resource_database_id) AS [database],
t1.resource_associated_entity_id AS [blk object],t1.request_mode AS [lock req],  -- lock requested
t1.request_session_id AS [waiter sid], t2.wait_duration_ms AS [wait time],       -- spid of waiter  
(SELECT [text] FROM sys.dm_exec_requests AS r WITH (NOLOCK)                      -- get sql for waiter
CROSS APPLY sys.dm_exec_sql_text(r.[sql_handle]) 
WHERE r.session_id = t1.request_session_id) AS [waiter_batch],
(SELECT SUBSTRING(qt.[text],r.statement_start_offset/2, 
    (CASE WHEN r.statement_end_offset = -1 
    THEN LEN(CONVERT(nvarchar(max), qt.[text])) * 2 
    ELSE r.statement_end_offset END - r.statement_start_offset)/2) 
FROM sys.dm_exec_requests AS r WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(r.[sql_handle]) AS qt
WHERE r.session_id = t1.request_session_id) AS [waiter_stmt],					-- statement blocked
t2.blocking_session_id AS [blocker sid],										-- spid of blocker
(SELECT [text] FROM sys.sysprocesses AS p										-- get sql for blocker
CROSS APPLY sys.dm_exec_sql_text(p.[sql_handle]) 
WHERE p.spid = t2.blocking_session_id) AS [blocker_batch]
FROM sys.dm_tran_locks AS t1 WITH (NOLOCK)
INNER JOIN sys.dm_os_waiting_tasks AS t2 WITH (NOLOCK)
ON t1.lock_owner_address = t2.resource_address OPTION (RECOMPILE);

(0 rows affected)

Total execution time: 00:00:00.078

lock type,database,blk object,lock req,waiter sid,wait time,waiter_batch,waiter_stmt,blocker sid,blocker_batch


Helps troubleshoot blocking and deadlocking issues
- The results will change from second to second on a busy system
- You should run this query multiple times when you see signs of blocking

## Find single-use, ad-hoc and prepared queries that are bloating the plan cache  (Query 13) (Ad hoc Queries)

In [34]:
-- Find single-use, ad-hoc and prepared queries that are bloating the plan cache  (Query 13) (Ad hoc Queries)
SELECT TOP(10) DB_NAME(t.[dbid]) AS [Database Name], t.[text] AS [Query Text], 
cp.objtype AS [Object Type], cp.cacheobjtype AS [Cache Object Type],  
cp.size_in_bytes/1024 AS [Plan Size in KB]
FROM sys.dm_exec_cached_plans AS cp WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t
WHERE cp.cacheobjtype = N'Compiled Plan' 
AND cp.objtype IN (N'Adhoc', N'Prepared') 
AND cp.usecounts = 1
ORDER BY cp.size_in_bytes DESC, DB_NAME(t.[dbid]) OPTION (RECOMPILE);

(10 rows affected)

Total execution time: 00:00:00.521

Database Name,Query Text,Object Type,Cache Object Type,Plan Size in KB
WideWorldImporters,"(@_msparam_0 nvarchar(4000))SELECT SCHEMA_NAME(udf.schema_id) AS [Schema], udf.name AS [Name], udf.object_id AS [ID], (case when 'FN' = udf.type then 1 when 'FS' = udf.type then 1 when 'IF' = udf.type then 3 when 'TF' = udf.type then 2 when 'FT' = udf.type then 2 else 0 end) AS [FunctionType], CASE WHEN udf.type IN ('FN','IF','TF') THEN 1 WHEN udf.type IN ('FS','FT') THEN 2 ELSE 1 END AS [ImplementationType], CAST( case when udf.is_ms_shipped = 1 then 1 else 0 end AS bit) AS [IsSystemObject], CAST(CASE WHEN ISNULL(smudf.definition, ssmudf.definition) IS NULL THEN 1 ELSE 0 END AS bit) AS [IsEncrypted], CAST(ISNULL(OBJECTPROPERTYEX(udf.object_id, N'IsSchemaBound'),0) AS bit) AS [IsSchemaBound], usrt.name AS [DataType], ISNULL(baset.name, N'') AS [SystemType], CAST(CASE WHEN baset.name IN (N'nchar', N'nvarchar') AND ret_param.max_length <> -1 THEN ret_param.max_length/2 ELSE ret_param.max_length END AS int) AS [Length], CAST(ret_param.precision AS int) AS [NumericPrecision], CAST(ret_param.scale AS int) AS [NumericScale], ISNULL(xscret_param.name, N'') AS [XmlSchemaNamespace], ISNULL(s2ret_param.name, N'') AS [XmlSchemaNamespaceSchema], ISNULL( (case ret_param.is_xml_document when 1 then 2 else 1 end), 0) AS [XmlDocumentConstraint], s1ret_param.name AS [DataTypeSchema] FROM sys.all_objects AS udf LEFT OUTER JOIN sys.sql_modules AS smudf ON smudf.object_id = udf.object_id LEFT OUTER JOIN sys.system_sql_modules AS ssmudf ON ssmudf.object_id = udf.object_id LEFT OUTER JOIN sys.all_parameters AS ret_param ON ret_param.object_id = udf.object_id and ret_param.is_output = @_msparam_0 LEFT OUTER JOIN sys.types AS usrt ON usrt.user_type_id = ret_param.user_type_id LEFT OUTER JOIN sys.types AS baset ON (baset.user_type_id = ret_param.system_type_id and baset.user_type_id = baset.system_type_id) or ((baset.system_type_id = ret_param.system_type_id) and (baset.user_type_id = ret_param.user_type_id) and (baset.is_user_defined = 0) and (baset.is_assembly_type = 1)) LEFT OUTER JOIN sys.xml_schema_collections AS xscret_param ON xscret_param.xml_collection_id = ret_param.xml_collection_id LEFT OUTER JOIN sys.schemas AS s2ret_param ON s2ret_param.schema_id = xscret_param.schema_id LEFT OUTER JOIN sys.schemas AS s1ret_param ON s1ret_param.schema_id = usrt.schema_id WHERE (udf.type in ('TF', 'FN', 'IF', 'FS', 'FT')) ORDER BY [Schema] ASC,[Name] ASC",Prepared,Compiled Plan,552
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4000))SELECT u.name AS [Name], u.principal_id AS [ID], CAST(CASE WHEN u.principal_id < 5 OR u.principal_id = 16382 OR u.principal_id = 16383 THEN 1 ELSE 0 END AS bit) AS [IsSystemObject], CASE WHEN N'U' = u.type THEN 0 WHEN N'G' = u.type THEN 1 WHEN N'S' = u.type THEN 2 WHEN N'C' = u.type THEN 3 WHEN N'K' = u.type THEN 4 WHEN N'E' = u.type THEN 5 WHEN N'X' = u.type THEN 6 END AS [LoginType], CAST(CASE dp.state WHEN N'G' THEN 1 WHEN 'W' THEN 1 ELSE 0 END AS bit) AS [HasDBAccess], u.sid AS [Sid], u.create_date AS [CreateDate], u.modify_date AS [DateLastModified], ISNULL(u.default_schema_name,N'') AS [DefaultSchema], ISNULL(ak.name,N'') AS [AsymmetricKey], ISNULL(cert.name,N'') AS [Certificate], CASE WHEN N'C' = u.type THEN 1 WHEN N'K' = u.type THEN 2 WHEN N'S' = u.type AND u.authentication_type != 2 AND u.authentication_type != 1 THEN 3 WHEN N'E' = u.type or N'X' = u.type THEN 4 ELSE 0 END AS [UserType], u.authentication_type AS [AuthenticationType] FROM sys.database_principals AS u LEFT OUTER JOIN sys.database_permissions AS dp ON dp.grantee_principal_id = u.principal_id and dp.type = @_msparam_0 LEFT OUTER JOIN sys.asymmetric_keys AS ak ON ak.sid = u.sid LEFT OUTER JOIN sys.certificates AS cert ON cert.sid = u.sid WHERE (u.type in ('U', 'S', 'G', 'C', 'K' ,'E', 'X'))and(u.name=@_msparam_1)",Prepared,Compiled Plan,400
master,"DECLARE @serverUtcDate DATETIME SET @serverUtcDate = CAST(SERVERPROPERTY('startutcdate') as DATET

Gives you the text, type and size of single-use ad-hoc and prepared queries that waste space in the plan cache
- Enabling 'optimize for ad hoc workloads' for the instance can help (SQL Server 2008 and above only)
- Running DBCC FREESYSTEMCACHE ('SQL Plans') periodically may be required to better control this
- Enabling forced parameterization for the database can help, but test first!

[Plan cache, adhoc workloads and clearing the single-use plan cache bloat](https://bit.ly/2EfYOkl)

# Database specific queries

> **Note**: Please switch to a user database that you are interested in!

In [35]:
-- **** Please switch to a user database that you are interested in! *****
--USE YourDatabaseName; -- make sure to change to an actual database on your instance, not the master system database
--GO

Commands completed successfully.

Total execution time: 00:00:00.458

## Individual File Sizes and space available for current database  (Query 15) (File Sizes and Space)

In [36]:
-- Individual File Sizes and space available for current database  (Query 15) (File Sizes and Space)
SELECT f.name AS [File Name] , f.physical_name AS [Physical Name], 
CAST((f.size/128.0) AS DECIMAL(15,2)) AS [Total Size in MB],
CAST(f.size/128.0 - CAST(FILEPROPERTY(f.name, 'SpaceUsed') AS int)/128.0 AS DECIMAL(15,2)) 
AS [Available Space In MB], f.[file_id], fg.name AS [Filegroup Name],
f.is_percent_growth, f.growth, fg.is_default, fg.is_read_only, 
fg.is_autogrow_all_files
FROM sys.database_files AS f WITH (NOLOCK) 
LEFT OUTER JOIN sys.filegroups AS fg WITH (NOLOCK)
ON f.data_space_id = fg.data_space_id
ORDER BY f.[file_id] OPTION (RECOMPILE);

(3 rows affected)

Total execution time: 00:00:00.091

File Name,Physical Name,Total Size in MB,Available Space In MB,file_id,Filegroup Name,is_percent_growth,growth,is_default,is_read_only,is_autogrow_all_files
data_0,1209ee52-887c-4af6-a30d-3e38cac61ff1_1.mdf,688.00,13.44,1,PRIMARY,0,2048,1,0,0
log,1209ee52-887c-4af6-a30d-3e38cac61ff1_2.ldf,1000.00,994.02,2,NULL,0,2048,NULL,NULL,NULL
XTP,7f375e7f-cd47-4c28-8924-ce9e7c543a4f.xtp,0.00,NULL,65537,NULL,0,0,NULL,NULL,NULL


Look at how large and how full the files are and where they are located
- Make sure the transaction log is not full!!
- is_autogrow_all_files was new for SQL Server 2016. Equivalent to TF 1117 for user databases

[SQL Server 2016: Changes in default behavior for autogrow and allocations for tempdb and user databases](https://bit.ly/2evRZSR)

## Log space usage for current database  (Query 16) (Log Space Usage)

In [37]:
-- Log space usage for current database  (Query 16) (Log Space Usage)
SELECT DB_NAME(lsu.database_id) AS [Database Name], db.recovery_model_desc AS [Recovery Model],
		CAST(lsu.total_log_size_in_bytes/1048576.0 AS DECIMAL(10, 2)) AS [Total Log Space (MB)],
		CAST(lsu.used_log_space_in_bytes/1048576.0 AS DECIMAL(10, 2)) AS [Used Log Space (MB)], 
		CAST(lsu.used_log_space_in_percent AS DECIMAL(10, 2)) AS [Used Log Space %],
		CAST(lsu.log_space_in_bytes_since_last_backup/1048576.0 AS DECIMAL(10, 2)) AS [Used Log Space Since Last Backup (MB)],
		db.log_reuse_wait_desc		 
FROM sys.dm_db_log_space_usage AS lsu WITH (NOLOCK)
INNER JOIN sys.databases AS db WITH (NOLOCK)
ON lsu.database_id = db.database_id
OPTION (RECOMPILE);

(1 row affected)

Total execution time: 00:00:00.134

Database Name,Recovery Model,Total Log Space (MB),Used Log Space (MB),Used Log Space %,Used Log Space Since Last Backup (MB),log_reuse_wait_desc
WideWorldImporters,FULL,999.99,5.98,0.60,0.30,NOTHING


Look at log file size and usage, along with the log reuse wait description for the current database

[sys.dm_db_log_space_usage (Transact-SQL)](https://bit.ly/2H4MQw9)

## Important database properties for current database   (Query 19) (Database Properties)

In [38]:
-- Important database properties for current database   (Query 19) (Database Properties)
SELECT db.[name] AS [Database Name], db.recovery_model_desc AS [Recovery Model], 
db.state_desc, db.containment_desc, db.log_reuse_wait_desc AS [Log Reuse Wait Description], 
db.[compatibility_level] AS [DB Compatibility Level], 
db.is_mixed_page_allocation_on, db.page_verify_option_desc AS [Page Verify Option], 
db.is_auto_create_stats_on, db.is_auto_update_stats_on, db.is_auto_update_stats_async_on, db.is_parameterization_forced, 
db.snapshot_isolation_state_desc, db.is_read_committed_snapshot_on, db.is_auto_close_on, db.is_auto_shrink_on, 
db.target_recovery_time_in_seconds, db.is_cdc_enabled, db.is_memory_optimized_elevate_to_snapshot_on, 
db.delayed_durability_desc, db.is_auto_create_stats_incremental_on,
db.is_query_store_on, db.is_sync_with_backup, db.is_temporal_history_retention_enabled,
db.is_encrypted, is_result_set_caching_on, is_accelerated_database_recovery_on, is_tempdb_spill_to_remote_store  
FROM sys.databases AS db WITH (NOLOCK)
WHERE db.[name] <> N'master'
ORDER BY db.[name] OPTION (RECOMPILE);

(1 row affected)

Total execution time: 00:00:00.301

Database Name,Recovery Model,state_desc,containment_desc,Log Reuse Wait Description,DB Compatibility Level,is_mixed_page_allocation_on,Page Verify Option,is_auto_create_stats_on,is_auto_update_stats_on,is_auto_update_stats_async_on,is_parameterization_forced,snapshot_isolation_state_desc,is_read_committed_snapshot_on,is_auto_close_on,is_auto_shrink_on,target_recovery_time_in_seconds,is_cdc_enabled,is_memory_optimized_elevate_to_snapshot_on,delayed_durability_desc,is_auto_create_stats_incremental_on,is_query_store_on,is_sync_with_backup,is_temporal_history_retention_enabled,is_encrypted,is_result_set_caching_on,is_accelerated_database_recovery_on,is_tempdb_spill_to_remote_store
WideWorldImporters,FULL,ONLINE,NONE,NOTHING,130,0,CHECKSUM,1,1,1,0,OFF,1,0,0,120,0,0,DISABLED,0,1,0,1,1,0,1,0


Things to look at:
- What recovery model are you using?
- What is the log reuse wait description?
- What compatibility level is the database on? 
- What is the Page Verify Option? (should be CHECKSUM)
- Is Auto Update Statistics Asynchronously enabled?
- Is Delayed Durability enabled?

## Get recent resource usage (Query 23) (Avg/Max Resource Usage)

In [39]:
-- Get recent resource usage (Query 23) (Avg/Max Resource Usage)
SELECT CAST(AVG(avg_cpu_percent) AS DECIMAL(10,2)) AS [Average CPU Utilization In Percent],   
       CAST(MAX(avg_cpu_percent) AS DECIMAL(10,2)) AS [Maximum CPU Utilization In Percent],   
       CAST(AVG(avg_data_io_percent) AS DECIMAL(10,2)) AS [Average Data IO In Percent],   
       CAST(MAX(avg_data_io_percent) AS DECIMAL(10,2)) AS [Maximum Data IO In Percent],   
       CAST(AVG(avg_log_write_percent) AS DECIMAL(10,2)) AS [Average Log Write Utilization In Percent],   
       CAST(MAX(avg_log_write_percent) AS DECIMAL(10,2)) AS [Maximum Log Write Utilization In Percent],   
       CAST(AVG(avg_memory_usage_percent) AS DECIMAL(10,2)) AS [Average Memory Usage In Percent],   
       CAST(MAX(avg_memory_usage_percent) AS DECIMAL(10,2)) AS [Maximum Memory Usage In Percent]   
FROM sys.dm_db_resource_stats WITH (NOLOCK) OPTION (RECOMPILE); 

(1 row affected)

Total execution time: 00:00:00.106

Average CPU Utilization In Percent,Maximum CPU Utilization In Percent,Average Data IO In Percent,Maximum Data IO In Percent,Average Log Write Utilization In Percent,Maximum Log Write Utilization In Percent,Average Memory Usage In Percent,Maximum Memory Usage In Percent
2.08,53.69,0.00,0.00,0.00,0.04,1.46,1.48


## Isolate top waits for this database since last restart or failover (Query 24) (Top DB Waits)

In [40]:
-- Isolate top waits for this database since last restart or failover (Query 24) (Top DB Waits)
WITH [Waits] 
AS (SELECT wait_type, wait_time_ms/ 1000.0 AS [WaitS],
          (wait_time_ms - signal_wait_time_ms) / 1000.0 AS [ResourceS],
           signal_wait_time_ms / 1000.0 AS [SignalS],
           waiting_tasks_count AS [WaitCount],
           100.0 *  wait_time_ms / SUM (wait_time_ms) OVER() AS [Percentage],
           ROW_NUMBER() OVER(ORDER BY wait_time_ms DESC) AS [RowNum]
    FROM sys.dm_db_wait_stats WITH (NOLOCK)
    WHERE [wait_type] NOT IN (
        N'BROKER_EVENTHANDLER', N'BROKER_RECEIVE_WAITFOR', N'BROKER_TASK_STOP',
		N'BROKER_TO_FLUSH', N'BROKER_TRANSMITTER', N'CHECKPOINT_QUEUE',
        N'CHKPT', N'CLR_AUTO_EVENT', N'CLR_MANUAL_EVENT', N'CLR_SEMAPHORE',
        N'DBMIRROR_DBM_EVENT', N'DBMIRROR_EVENTS_QUEUE', N'DBMIRROR_WORKER_QUEUE',
		N'DBMIRRORING_CMD', N'DIRTY_PAGE_POLL', N'DISPATCHER_QUEUE_SEMAPHORE',
        N'EXECSYNC', N'FSAGENT', N'FT_IFTS_SCHEDULER_IDLE_WAIT', N'FT_IFTSHC_MUTEX',
        N'HADR_CLUSAPI_CALL', N'HADR_FILESTREAM_IOMGR_IOCOMPLETION', N'HADR_LOGCAPTURE_WAIT', 
		N'HADR_NOTIFICATION_DEQUEUE', N'HADR_TIMER_TASK', N'HADR_WORK_QUEUE',
        N'KSOURCE_WAKEUP', N'LAZYWRITER_SLEEP', N'LOGMGR_QUEUE', 
		N'MEMORY_ALLOCATION_EXT', N'ONDEMAND_TASK_QUEUE',
		N'PREEMPTIVE_HADR_LEASE_MECHANISM', N'PREEMPTIVE_SP_SERVER_DIAGNOSTICS',
		N'PREEMPTIVE_ODBCOPS',
		N'PREEMPTIVE_OS_LIBRARYOPS', N'PREEMPTIVE_OS_COMOPS', N'PREEMPTIVE_OS_CRYPTOPS',
		N'PREEMPTIVE_OS_PIPEOPS', N'PREEMPTIVE_OS_AUTHENTICATIONOPS',
		N'PREEMPTIVE_OS_GENERICOPS', N'PREEMPTIVE_OS_VERIFYTRUST',
		N'PREEMPTIVE_OS_FILEOPS', N'PREEMPTIVE_OS_DEVICEOPS', N'PREEMPTIVE_OS_QUERYREGISTRY',
		N'PREEMPTIVE_OS_WRITEFILE',
		N'PREEMPTIVE_XE_CALLBACKEXECUTE', N'PREEMPTIVE_XE_DISPATCHER',
		N'PREEMPTIVE_XE_GETTARGETSTATE', N'PREEMPTIVE_XE_SESSIONCOMMIT',
		N'PREEMPTIVE_XE_TARGETINIT', N'PREEMPTIVE_XE_TARGETFINALIZE',
		N'PREEMPTIVE_XHTTP',
        N'PWAIT_ALL_COMPONENTS_INITIALIZED', N'PWAIT_DIRECTLOGCONSUMER_GETNEXT',
		N'QDS_PERSIST_TASK_MAIN_LOOP_SLEEP',
		N'QDS_ASYNC_QUEUE',
        N'QDS_CLEANUP_STALE_QUERIES_TASK_MAIN_LOOP_SLEEP', N'REQUEST_FOR_DEADLOCK_SEARCH',
		N'RESOURCE_GOVERNOR_IDLE',
		N'RESOURCE_QUEUE', N'SERVER_IDLE_CHECK', N'SLEEP_BPOOL_FLUSH', N'SLEEP_DBSTARTUP',
		N'SLEEP_DCOMSTARTUP', N'SLEEP_MASTERDBREADY', N'SLEEP_MASTERMDREADY',
        N'SLEEP_MASTERUPGRADED', N'SLEEP_MSDBSTARTUP', N'SLEEP_SYSTEMTASK', N'SLEEP_TASK',
        N'SLEEP_TEMPDBSTARTUP', N'SNI_HTTP_ACCEPT', N'SP_SERVER_DIAGNOSTICS_SLEEP',
		N'SQLTRACE_BUFFER_FLUSH', N'SQLTRACE_INCREMENTAL_FLUSH_SLEEP', N'SQLTRACE_WAIT_ENTRIES',
		N'WAIT_FOR_RESULTS', N'WAITFOR', N'WAITFOR_TASKSHUTDOWN', N'WAIT_XTP_HOST_WAIT',
		N'WAIT_XTP_OFFLINE_CKPT_NEW_LOG', N'WAIT_XTP_CKPT_CLOSE', N'WAIT_XTP_RECOVERY',
		N'XE_BUFFERMGR_ALLPROCESSED_EVENT', N'XE_DISPATCHER_JOIN',
        N'XE_DISPATCHER_WAIT', N'XE_LIVE_TARGET_TVF', N'XE_TIMER_EVENT')
    AND waiting_tasks_count > 0)
SELECT
    MAX (W1.wait_type) AS [WaitType],
	CAST (MAX (W1.Percentage) AS DECIMAL (5,2)) AS [Wait Percentage],
	CAST ((MAX (W1.WaitS) / MAX (W1.WaitCount)) AS DECIMAL (16,4)) AS [AvgWait_Sec],
    CAST ((MAX (W1.ResourceS) / MAX (W1.WaitCount)) AS DECIMAL (16,4)) AS [AvgRes_Sec],
    CAST ((MAX (W1.SignalS) / MAX (W1.WaitCount)) AS DECIMAL (16,4)) AS [AvgSig_Sec],
    CAST (MAX (W1.WaitS) AS DECIMAL (16,2)) AS [Total_Wait_Sec],
    CAST (MAX (W1.ResourceS) AS DECIMAL (16,2)) AS [Resource_Sec],
    CAST (MAX (W1.SignalS) AS DECIMAL (16,2)) AS [Signal_Sec],
    MAX (W1.WaitCount) AS [Wait Count]   
FROM Waits AS W1
INNER JOIN Waits AS W2
ON W2.RowNum <= W1.RowNum
GROUP BY W1.RowNum
HAVING SUM (W2.Percentage) - MAX (W1.Percentage) < 99 -- percentage threshold
OPTION (RECOMPILE);

(4 rows affected)

Total execution time: 00:00:00.137

WaitType,Wait Percentage,AvgWait_Sec,AvgRes_Sec,AvgSig_Sec,Total_Wait_Sec,Resource_Sec,Signal_Sec,Wait Count
SOS_SCHEDULER_YIELD,73.47,0.0111,0.0000,0.0111,50.20,0.00,50.20,4507
PAGEIOLATCH_SH,14.34,0.0265,0.0260,0.0005,9.80,9.60,0.19,370
PWAIT_SECURITY_FEDAUTH_AADLOOKUP,10.67,1.2148,1.2148,0.0000,7.29,7.29,0.00,6
WRITELOG,1.10,0.0142,0.0138,0.0003,0.75,0.73,0.02,53


Cumulative wait stats are not as useful on an idle instance that is not under load or performance pressure

[SQL Server Wait Types Library](https://bit.ly/2ePzYO2)

[The SQL Server Wait Type Repository](https://bit.ly/1afzfjC)

[Wait statistics, or please tell me where it hurts](https://bit.ly/2wsQHQE)

[SQL Server 2005 Performance Tuning using the Waits and Queues](https://bit.ly/1o2NFoF)

[sys.dm_os_wait_stats (Transact-SQL)](https://bit.ly/2Hjq9Yl)


## Get most frequently executed queries for this database (Query 25) (Query Execution Counts)

In [41]:
-- Get most frequently executed queries for this database (Query 25) (Query Execution Counts)
SELECT TOP(5) LEFT(t.[text], 50) AS [Short Query Text], qs.execution_count AS [Execution Count],
qs.total_logical_reads AS [Total Logical Reads],
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads],
qs.total_worker_time AS [Total Worker Time],
qs.total_worker_time/qs.execution_count AS [Avg Worker Time], 
qs.total_elapsed_time AS [Total Elapsed Time],
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time],
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) COLLATE Latin1_General_BIN2 LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],
qs.creation_time AS [Creation Time]
--,t.[text] AS [Complete Query Text], qp.query_plan AS [Query Plan] -- uncomment out these columns if not copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp 
WHERE t.dbid = DB_ID()
ORDER BY qs.execution_count DESC OPTION (RECOMPILE);

(5 rows affected)

Total execution time: 00:00:00.085

Short Query Text,Execution Count,Total Logical Reads,Avg Logical Reads,Total Worker Time,Avg Worker Time,Total Elapsed Time,Avg Elapsed Time,Has Missing Index,Creation Time
"(@ip_address_value BIGINT, @is_present INT OUT) DE",392,0,0,15498,39,15661,39,0,2021-04-30 00:53:09.193
"(@ip_address_value BIGINT, @is_present INT OUT) DE",392,0,0,6586,16,6651,16,0,2021-04-30 00:53:09.197
"SELECT SERVERPROPERTY('EngineEdition'), SERVERPROP",165,1155,7,44125,267,44226,268,0,2021-04-30 00:53:09.397
"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",34,15607,459,62138,1827,62153,1828,0,2021-04-30 01:46:00.223
"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",15,264,17,6042,402,6049,403,0,2021-04-30 01:47:26.520


Tells you which cached queries are called the most often
- This helps you characterize and baseline your workload
- It also helps you find possible caching opportunities

## Get top total worker time queries for this database (Query 26) (Top Worker Time Queries)

In [42]:
-- Get top total worker time queries for this database (Query 26) (Top Worker Time Queries)		
SELECT TOP(5) DB_NAME(t.[dbid]) AS [Database Name], 
REPLACE(REPLACE(LEFT(t.[text], 50), CHAR(10),''), CHAR(13),'') AS [Short Query Text],  
qs.total_worker_time AS [Total Worker Time], qs.min_worker_time AS [Min Worker Time],
qs.total_worker_time/qs.execution_count AS [Avg Worker Time], 
qs.max_worker_time AS [Max Worker Time], 
qs.min_elapsed_time AS [Min Elapsed Time], 
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time], 
qs.max_elapsed_time AS [Max Elapsed Time],
qs.min_logical_reads AS [Min Logical Reads],
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads],
qs.max_logical_reads AS [Max Logical Reads], 
qs.execution_count AS [Execution Count],
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],  
qs.creation_time AS [Creation Time]
--,t.[text] AS [Query Text], qp.query_plan AS [Query Plan] -- uncomment out these columns if not copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp
WHERE t.dbid = DB_ID() 
ORDER BY qs.total_worker_time DESC OPTION (RECOMPILE);

(5 rows affected)

Total execution time: 00:00:00.648

Database Name,Short Query Text,Total Worker Time,Min Worker Time,Avg Worker Time,Max Worker Time,Min Elapsed Time,Avg Elapsed Time,Max Elapsed Time,Min Logical Reads,Avg Logical Reads,Max Logical Reads,Execution Count,Has Missing Index,Creation Time
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",125460,16433,20910,23966,16433,70040,238141,194,194,195,6,0,2021-04-30 01:47:25.690
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",62138,579,1827,4546,579,1828,4546,47,459,1598,34,0,2021-04-30 01:46:00.223
WideWorldImporters,"SELECT SERVERPROPERTY('EngineEdition'), SERVERPROP",44317,139,266,485,139,267,486,7,7,7,166,0,2021-04-30 00:53:09.397
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",42180,42180,42180,42180,44213,44213,44213,16780,16780,16780,1,0,2021-04-30 01:45:35.020
WideWorldImporters,(@_msparam_0 nvarchar(4000))SELECTSCHEMA_NAME(udf,29028,29028,29028,29028,154729,154729,154729,4717,4717,4717,1,0,2021-04-30 01:45:33.370


- Helps you find the most expensive queries from a CPU perspective for this database
- Can also help track down parameter sniffing issues

## Get top 5 total logical reads queries for this database (Query 27) (Top Logical Reads Queries)

In [43]:
SELECT TOP(5) DB_NAME(t.[dbid]) AS [Database Name],
REPLACE(REPLACE(LEFT(t.[text], 50), CHAR(10),''), CHAR(13),'') AS [Short Query Text], 
qs.total_logical_reads AS [Total Logical Reads],
qs.min_logical_reads AS [Min Logical Reads],
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads],
qs.max_logical_reads AS [Max Logical Reads],   
qs.min_worker_time AS [Min Worker Time],
qs.total_worker_time/qs.execution_count AS [Avg Worker Time], 
qs.max_worker_time AS [Max Worker Time], 
qs.min_elapsed_time AS [Min Elapsed Time], 
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time], 
qs.max_elapsed_time AS [Max Elapsed Time],
qs.execution_count AS [Execution Count],
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],   
qs.creation_time AS [Creation Time]
--,t.[text] AS [Complete Query Text], qp.query_plan AS [Query Plan] -- uncomment out these columns if not copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp
WHERE t.dbid = DB_ID()  
ORDER BY qs.total_logical_reads DESC OPTION (RECOMPILE);

(5 rows affected)

Total execution time: 00:00:00.164

Database Name,Short Query Text,Total Logical Reads,Min Logical Reads,Avg Logical Reads,Max Logical Reads,Min Worker Time,Avg Worker Time,Max Worker Time,Min Elapsed Time,Avg Elapsed Time,Max Elapsed Time,Execution Count,Has Missing Index,Creation Time
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",16780,16780,16780,16780,42180,42180,42180,44213,44213,44213,1,0,2021-04-30 01:45:35.020
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",15607,47,459,1598,579,1827,4546,579,1828,4546,34,0,2021-04-30 01:46:00.223
WideWorldImporters,(@_msparam_0 nvarchar(4000))SELECTSCHEMA_NAME(udf,4717,4717,4717,4717,29028,29028,29028,154729,154729,154729,1,0,2021-04-30 01:45:33.370
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4",1720,59,286,831,678,1433,2621,678,5788,27897,6,0,2021-04-30 01:47:26.927
WideWorldImporters,"SELECT SERVERPROPERTY('EngineEdition'), SERVERPROP",1169,7,7,7,139,267,485,139,267,486,167,0,2021-04-30 00:53:09.397


- Helps you find the most expensive queries from a memory perspective for this database
- Can also help track down parameter sniffing issues

## Get top average elapsed time queries for this database (Query 28) (Top Avg Elapsed Time Queries)

In [44]:
SELECT TOP(5) DB_NAME(t.[dbid]) AS [Database Name], 
REPLACE(REPLACE(LEFT(t.[text], 255), CHAR(10),''), CHAR(13),'') AS [Short Query Text],  
qs.total_elapsed_time/qs.execution_count AS [Avg Elapsed Time],
qs.min_elapsed_time, qs.max_elapsed_time, qs.last_elapsed_time,
qs.execution_count AS [Execution Count],  
qs.total_logical_reads/qs.execution_count AS [Avg Logical Reads], 
qs.total_physical_reads/qs.execution_count AS [Avg Physical Reads], 
qs.total_worker_time/qs.execution_count AS [Avg Worker Time],
CASE WHEN CONVERT(nvarchar(max), qp.query_plan) LIKE N'%<MissingIndexes>%' THEN 1 ELSE 0 END AS [Has Missing Index],  
qs.creation_time AS [Creation Time]
-- , qp.query_plan AS [Query Plan] -- comment out this column if copying results to Excel
FROM sys.dm_exec_query_stats AS qs WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) AS t 
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp
WHERE t.dbid = DB_ID()  
ORDER BY qs.total_elapsed_time/qs.execution_count DESC OPTION (RECOMPILE);

(5 rows affected)

Total execution time: 00:00:01.430

Database Name,Short Query Text,Avg Elapsed Time,min_elapsed_time,max_elapsed_time,last_elapsed_time,Execution Count,Avg Logical Reads,Avg Physical Reads,Avg Worker Time,Has Missing Index,Creation Time
WideWorldImporters,"(@_msparam_0 nvarchar(4000))SELECTSCHEMA_NAME(udf.schema_id) AS [Schema],udf.name AS [Name],udf.object_id AS [ID],(case when 'FN' = udf.type then 1 when 'FS' = udf.type then 1 when 'IF' = udf.type then 3 when 'TF' = udf.type then 2 when 'FT' = udf.typ",154729,154729,154729,154729,1,4717,0,29028,0,2021-04-30 01:45:33.370
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4000),@_msparam_2 nvarchar(4000))SELECTudf.name AS [Name],udf.object_id AS [ID],udf.create_date AS [CreateDate],udf.modify_date AS [DateLastModified],ISNULL(sudf.name, N'') AS [Owner],CAST(case when u",70040,16433,238141,22644,6,194,0,20910,0,2021-04-30 01:47:25.690
WideWorldImporters,"(@_msparam_0 nvarchar(4000),@_msparam_1 nvarchar(4000),@_msparam_2 nvarchar(4000))SELECTSCHEMA_NAME(sp.schema_id) AS [Schema],sp.name AS [Name],sp.object_id AS [ID],CAST( case when sp.is_ms_shipped = 1 then 1 else 0",44213,44213,44213,44213,1,16780,0,42180,0,2021-04-30 01:45:35.020
WideWorldImporters,"SELECTtr.name AS [Name],tr.object_id AS [ID],CAST( tr.is_ms_shipped AS bit) AS [IsSystemObject],CASE WHEN tr.type = N'TR' THEN 1 WHEN tr.type = N'TA' THEN 2 ELSE 1 END AS [ImplementationType],CAST(CASE WHEN ISNULL(smtr.d",28407,28407,28407,28407,1,79,0,1385,0,2021-04-30 01:45:26.083
WideWorldImporters,"SELECTSCHEMA_NAME(tbl.schema_id) AS [Schema],tbl.name AS [Name],tbl.object_id AS [ID]FROMsys.tables AS tblORDER BY[Schema] ASC,[Name] ASC",15112,15112,15112,15112,1,525,0,3462,0,2021-04-30 01:45:30.190


- Helps you find the highest average elapsed time queries for this database
- Can also help track down parameter sniffing issues

## Missing Indexes for current database by Index Advantage  (Query 38) (Missing Indexes)

In [45]:
-- Missing Indexes for current database by Index Advantage  (Query 38) (Missing Indexes)
SELECT CONVERT(decimal(18,2), migs.user_seeks * migs.avg_total_user_cost * (migs.avg_user_impact * 0.01)) AS [index_advantage], 
FORMAT(migs.last_user_seek, 'yyyy-MM-dd HH:mm:ss') AS [last_user_seek], mid.[statement] AS [Database.Schema.Table], 
COUNT(1) OVER(PARTITION BY mid.[statement]) AS [missing_indexes_for_table], 
COUNT(1) OVER(PARTITION BY mid.[statement], mid.equality_columns) AS [similar_missing_indexes_for_table], 
mid.equality_columns, mid.inequality_columns, mid.included_columns, migs.user_seeks, 
CONVERT(decimal(18,2), migs.avg_total_user_cost) AS [avg_total_user_,cost], migs.avg_user_impact,
REPLACE(REPLACE(LEFT(st.[text], 255), CHAR(10),''), CHAR(13),'') AS [Short Query Text],
OBJECT_NAME(mid.[object_id]) AS [Table Name], p.rows AS [Table Rows]
FROM sys.dm_db_missing_index_groups AS mig WITH (NOLOCK) 
INNER JOIN sys.dm_db_missing_index_group_stats_query AS migs WITH(NOLOCK) 
ON mig.index_group_handle = migs.group_handle 
CROSS APPLY sys.dm_exec_sql_text(migs.last_sql_handle) AS st 
INNER JOIN sys.dm_db_missing_index_details AS mid WITH (NOLOCK) 
ON mig.index_handle = mid.index_handle
INNER JOIN sys.partitions AS p WITH (NOLOCK)
ON p.[object_id] = mid.[object_id]
WHERE mid.database_id = DB_ID()
AND p.index_id < 2 
ORDER BY index_advantage DESC OPTION (RECOMPILE);

(0 rows affected)

Total execution time: 00:00:00.155

index_advantage,last_user_seek,Database.Schema.Table,missing_indexes_for_table,similar_missing_indexes_for_table,equality_columns,inequality_columns,included_columns,user_seeks,"avg_total_user_,cost",avg_user_impact,Short Query Text,Table Name,Table Rows


Look at index advantage, last user seek time, number of user seeks to help determine source and importance
- SQL Server is overly eager to add included columns, so beware
- Do not just blindly add indexes that show up from this query!!!

Håkan Winther has given me some great suggestions for this query

## Find missing index warnings for cached plans in the current database  (Query 39) (Missing Index Warnings)

In [46]:
-- Find missing index warnings for cached plans in the current database  (Query 39) (Missing Index Warnings)
-- Note: This query could take some time on a busy instance
SELECT TOP(25) OBJECT_NAME(objectid) AS [ObjectName], 
               cp.objtype, cp.usecounts, cp.size_in_bytes, qp.query_plan
FROM sys.dm_exec_cached_plans AS cp WITH (NOLOCK)
CROSS APPLY sys.dm_exec_query_plan(cp.plan_handle) AS qp
WHERE CAST(query_plan AS NVARCHAR(MAX)) LIKE N'%MissingIndex%'
AND dbid = DB_ID()
ORDER BY cp.usecounts DESC OPTION (RECOMPILE);

(0 rows affected)

Total execution time: 00:00:07.428

ObjectName,objtype,usecounts,size_in_bytes,query_plan


Helps you connect missing indexes to specific stored procedures or queries
- This can help you decide whether to add them or not

## Get Table names, row counts, and compression status for clustered index or heap  (Query 41) (Table Sizes)

In [47]:
-- Get Table names, row counts, and compression status for clustered index or heap  (Query 41) (Table Sizes)
SELECT TOP 10 SCHEMA_NAME(o.Schema_ID) AS [Schema Name], OBJECT_NAME(p.object_id) AS [ObjectName], 
SUM(p.Rows) AS [RowCount], p.data_compression_desc AS [Compression Type]
FROM sys.partitions AS p WITH (NOLOCK)
INNER JOIN sys.objects AS o WITH (NOLOCK)
ON p.object_id = o.object_id
WHERE index_id < 2 --ignore the partitions from the non-clustered index if any
AND OBJECT_NAME(p.object_id) NOT LIKE N'sys%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'spt_%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'queue_%' 
AND OBJECT_NAME(p.object_id) NOT LIKE N'filestream_tombstone%' 
AND OBJECT_NAME(p.object_id) NOT LIKE N'fulltext%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'ifts_comp_fragment%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'filetable_updates%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'xml_index_nodes%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'sqlagent_job%'
AND OBJECT_NAME(p.object_id) NOT LIKE N'plan_persist%'
GROUP BY  SCHEMA_NAME(o.Schema_ID), p.object_id, data_compression_desc
ORDER BY SUM(p.Rows) DESC OPTION (RECOMPILE);

(10 rows affected)

Total execution time: 00:00:00.115

Schema Name,ObjectName,RowCount,Compression Type
Warehouse,ColdRoomTemperatures_Archive,3654736,NONE
Warehouse,StockItemTransactions,236667,NONE
Sales,OrderLines,231412,NONE
Sales,InvoiceLines,228265,NONE
Sales,CustomerTransactions,97147,NONE
Sales,Orders,73595,NONE
Sales,Invoices,70510,NONE
Warehouse,VehicleTemperatures,65998,NONE
Application,Cities,37940,NONE
Purchasing,PurchaseOrderLines,8367,NONE


Gives you an idea of table sizes, and possible data compression opportunities

## When were Statistics last updated on for 10 oldest ones? (Query 43) (Statistics Update)

In [48]:
-- When were Statistics last updated on all indexes?  (Query 43) (Statistics Update)
SELECT TOP 10  SCHEMA_NAME(o.Schema_ID) + N'.' + o.[NAME] AS [Object Name], o.[type_desc] AS [Object Type],
      i.[name] AS [Index Name], STATS_DATE(i.[object_id], i.index_id) AS [Statistics Date], 
      s.auto_created, s.no_recompute, s.user_created, s.is_incremental, s.is_temporary,
	  st.row_count, st.used_page_count
FROM sys.objects AS o WITH (NOLOCK)
INNER JOIN sys.indexes AS i WITH (NOLOCK)
ON o.[object_id] = i.[object_id]
INNER JOIN sys.stats AS s WITH (NOLOCK)
ON i.[object_id] = s.[object_id] 
AND i.index_id = s.stats_id
INNER JOIN sys.dm_db_partition_stats AS st WITH (NOLOCK)
ON o.[object_id] = st.[object_id]
AND i.[index_id] = st.[index_id]
WHERE o.[type] IN ('U', 'V')
AND st.row_count > 0
AND STATS_DATE(i.[object_id], i.index_id) < DATEADD(DAY, -10, GETDATE())
ORDER BY STATS_DATE(i.[object_id], i.index_id) ASC OPTION (RECOMPILE);

(10 rows affected)

Total execution time: 00:00:00.383

Object Name,Object Type,Index Name,Statistics Date,auto_created,no_recompute,user_created,is_incremental,is_temporary,row_count,used_page_count
Application.Cities,USER_TABLE,FK_Application_Cities_StateProvinceID,2020-10-24 06:27:51.327,0,0,0,0,0,37940,138
Application.Countries,USER_TABLE,UQ_Application_Countries_CountryName,2020-10-24 06:27:51.360,0,0,0,0,0,190,4
Application.Countries,USER_TABLE,UQ_Application_Countries_FormalName,2020-10-24 06:27:51.380,0,0,0,0,0,190,4
Application.DeliveryMethods,USER_TABLE,UQ_Application_DeliveryMethods_DeliveryMethodName,2020-10-24 06:27:51.430,0,0,0,0,0,10,2
Application.PaymentMethods,USER_TABLE,UQ_Application_PaymentMethods_PaymentMethodName,2020-10-24 06:27:51.443,0,0,0,0,0,4,2
Application.People,USER_TABLE,IX_Application_People_FullName,2020-10-24 06:27:51.590,0,0,0,0,0,1111,10
Application.People,USER_TABLE,IX_Application_People_IsEmployee,2020-10-24 06:27:51.853,0,0,0,0,0,1111,6
Application.People,USER_TABLE,IX_Application_People_IsSalesperson,2020-10-24 06:27:51.877,0,0,0,0,0,1111,6
Application.People,USER_TABLE,IX_Application_People_Perf_20160301_05,2020-10-24 06:27:52.120,0,0,0,0,0,1111,17
Application.StateProvinces,USER_TABLE,FK_Application_StateProvinces_CountryID,2020-10-24 06:27:52.163,0,0,0,0,0,53,2


Helps discover possible problems with out-of-date statistics
- Also gives you an idea which indexes are the most active

[sys.stats (Transact-SQL)](https://bit.ly/2GyAxrn)

[UPDATEs to Statistics (Erin Stellato)](https://bit.ly/2vhrYQy)

## Look at most frequently modified indexes and statistics (Query 44) (Volatile Indexes)

In [49]:
-- Look at most frequently modified indexes and statistics (Query 44) (Volatile Indexes)
SELECT o.[name] AS [Object Name], o.[object_id], o.[type_desc], s.[name] AS [Statistics Name], 
       s.stats_id, s.no_recompute, s.auto_created, s.is_incremental, s.is_temporary,
	   sp.modification_counter, sp.[rows], sp.rows_sampled, sp.last_updated
FROM sys.objects AS o WITH (NOLOCK)
INNER JOIN sys.stats AS s WITH (NOLOCK)
ON s.object_id = o.object_id
CROSS APPLY sys.dm_db_stats_properties(s.object_id, s.stats_id) AS sp
WHERE o.[type_desc] NOT IN (N'SYSTEM_TABLE', N'INTERNAL_TABLE')
AND sp.modification_counter > 0
ORDER BY sp.modification_counter DESC, o.name OPTION (RECOMPILE);

(0 rows affected)

Total execution time: 00:00:00.480

Object Name,object_id,type_desc,Statistics Name,stats_id,no_recompute,auto_created,is_incremental,is_temporary,modification_counter,rows,rows_sampled,last_updated


This helps you understand your workload and make better decisions about things like data compression and adding new indexes to a table

## Get fragmentation info for all indexes above a certain size in the current database  (Query 45) (Index Fragmentation)

In [50]:
-- Get fragmentation info for all indexes above a certain size in the current database  (Query 45) (Index Fragmentation)
-- Note: This query could take some time on a very large database
SELECT DB_NAME(ps.database_id) AS [Database Name], SCHEMA_NAME(o.[schema_id]) AS [Schema Name],
OBJECT_NAME(ps.OBJECT_ID) AS [Object Name], i.[name] AS [Index Name], ps.index_id, 
ps.index_type_desc, ps.avg_fragmentation_in_percent, 
ps.fragment_count, ps.page_count, i.fill_factor, i.has_filter, 
i.filter_definition, i.[allow_page_locks]
FROM sys.dm_db_index_physical_stats(DB_ID(),NULL, NULL, NULL , N'LIMITED') AS ps
INNER JOIN sys.indexes AS i WITH (NOLOCK)
ON ps.[object_id] = i.[object_id] 
AND ps.index_id = i.index_id
INNER JOIN sys.objects AS o WITH (NOLOCK)
ON i.[object_id] = o.[object_id]
WHERE ps.database_id = DB_ID()
AND ps.page_count > 2500
ORDER BY ps.avg_fragmentation_in_percent DESC OPTION (RECOMPILE);

(5 rows affected)

Total execution time: 00:00:00.231

Database Name,Schema Name,Object Name,Index Name,index_id,index_type_desc,avg_fragmentation_in_percent,fragment_count,page_count,fill_factor,has_filter,filter_definition,allow_page_locks
WideWorldImporters,Sales,InvoiceLines,PK_Sales_InvoiceLines,1,CLUSTERED INDEX,0.27824151363383415,41,5391,0,0,NULL,1
WideWorldImporters,Warehouse,ColdRoomTemperatures_Archive,ix_ColdRoomTemperatures_Archive,1,CLUSTERED INDEX,0.2354413780771294,684,33554,0,0,NULL,1
WideWorldImporters,Warehouse,VehicleTemperatures,PK_Warehouse_VehicleTemperatures,1,CLUSTERED INDEX,0.23166023166023164,24,3885,0,0,NULL,1
WideWorldImporters,Sales,Invoices,PK_Sales_Invoices,1,CLUSTERED INDEX,0.21703051193667816,58,7833,0,0,NULL,1
WideWorldImporters,Sales,OrderLines,PK_Sales_OrderLines,1,CLUSTERED INDEX,0.17917579135974518,29,5023,0,0,NULL,1


Helps determine whether you have framentation in your relational indexes and how effective your index maintenance strategy is.


## Index Read/Write stats (TOP 20 in current DB) ordered by Reads (Query 46) (Overall Index Usage - Reads)

In [51]:
--- Index Read/Write stats (all tables in current DB) ordered by Reads  (Query 46) (Overall Index Usage - Reads)
SELECT top 20 
		OBJECT_NAME(i.[object_id]) AS [ObjectName], i.[name] AS [IndexName], i.index_id, 
       s.user_seeks, s.user_scans, s.user_lookups,
	   s.user_seeks + s.user_scans + s.user_lookups AS [Total Reads], 
	   s.user_updates AS [Writes],  
	   i.[type_desc] AS [Index Type], i.fill_factor AS [Fill Factor], i.has_filter, i.filter_definition, 
	   s.last_user_scan, s.last_user_lookup, s.last_user_seek
FROM sys.indexes AS i WITH (NOLOCK)
LEFT OUTER JOIN sys.dm_db_index_usage_stats AS s WITH (NOLOCK)
ON i.[object_id] = s.[object_id]
AND i.index_id = s.index_id
AND s.database_id = DB_ID()
WHERE OBJECTPROPERTY(i.[object_id],'IsUserTable') = 1
AND ISNULL(s.user_seeks + s.user_scans + s.user_lookups, 0) > 0
ORDER BY s.user_seeks + s.user_scans + s.user_lookups DESC OPTION (RECOMPILE); -- Order by reads

(0 rows affected)

Total execution time: 00:00:00.123

ObjectName,IndexName,index_id,user_seeks,user_scans,user_lookups,Total Reads,Writes,Index Type,Fill Factor,has_filter,filter_definition,last_user_scan,last_user_lookup,last_user_seek


Show which indexes in the current database are most active for Reads

## Index Read/Write stats (TOP 20 in current DB) ordered by Writes (Query 47) (Overall Index Usage - Writes)

In [52]:
--- Index Read/Write stats (all tables in current DB) ordered by Writes  (Query 47) (Overall Index Usage - Writes)
SELECT TOP 20 OBJECT_NAME(i.[object_id]) AS [ObjectName], i.[name] AS [IndexName], i.index_id,
	   s.user_updates AS [Writes], s.user_seeks + s.user_scans + s.user_lookups AS [Total Reads], 
	   i.[type_desc] AS [Index Type], i.fill_factor AS [Fill Factor], i.has_filter, i.filter_definition,
	   s.last_system_update, s.last_user_update
FROM sys.indexes AS i WITH (NOLOCK)
LEFT OUTER JOIN sys.dm_db_index_usage_stats AS s WITH (NOLOCK)
ON i.[object_id] = s.[object_id]
AND i.index_id = s.index_id
AND s.database_id = DB_ID()
WHERE OBJECTPROPERTY(i.[object_id],'IsUserTable') = 1
AND (s.user_seeks > 0 OR s.user_scans > 0 OR s.user_lookups > 0)
ORDER BY s.user_updates DESC OPTION (RECOMPILE);						 -- Order by writes

(0 rows affected)

Total execution time: 00:00:00.121

ObjectName,IndexName,index_id,Writes,Total Reads,Index Type,Fill Factor,has_filter,filter_definition,last_system_update,last_user_update


Show which indexes in the current database are most active for Writes

## Get in-memory OLTP index usage (Query 48) (XTP Index Usage)

In [53]:
-- Get in-memory OLTP index usage (Query 48) (XTP Index Usage)
SELECT OBJECT_NAME(i.[object_id]) AS [Object Name], i.index_id, i.[name] AS [Index Name],
       i.[type_desc], xis.scans_started, xis.scans_retries, 
	   xis.rows_touched, xis.rows_returned
FROM sys.dm_db_xtp_index_stats AS xis WITH (NOLOCK)
INNER JOIN sys.indexes AS i WITH (NOLOCK)
ON i.[object_id] = xis.[object_id] 
AND i.index_id = xis.index_id 
ORDER BY OBJECT_NAME(i.[object_id]) OPTION (RECOMPILE);

(0 rows affected)

Total execution time: 00:00:00.060

Object Name,index_id,Index Name,type_desc,scans_started,scans_retries,rows_touched,rows_returned


This gives you some index usage statistics for in-memory OLTP
- Returns no data if you are not using in-memory OLTP

[Guidelines for Using Indexes on Memory-Optimized Tables](https://bit.ly/2GCP8lF)

## Get database automatic tuning options (Query 56) (Automatic Tuning Options)

In [54]:
-- Get database automatic tuning options (Query 56) (Automatic Tuning Options)
SELECT [name], desired_state_desc, actual_state_desc, reason_desc
FROM sys.database_automatic_tuning_options WITH (NOLOCK)
OPTION (RECOMPILE);

(4 rows affected)

Total execution time: 00:00:00.046

name,desired_state_desc,actual_state_desc,reason_desc
FORCE_LAST_GOOD_PLAN,DEFAULT,ON,INHERITED_FROM_SERVER
CREATE_INDEX,DEFAULT,OFF,INHERITED_FROM_SERVER
DROP_INDEX,DEFAULT,OFF,INHERITED_FROM_SERVER
MAINTAIN_INDEX,DEFAULT,OFF,INHERITED_FROM_SERVER


[sys.database_automatic_tuning_options (Transact-SQL)](https://bit.ly/2FHhLkL)

## Retrieve some Azure SQL Database properties (Query 58) (Azure SQL DB Properties)

In [55]:
-- Retrieve some Azure SQL Database properties (Query 58) (Azure SQL DB Properties)
SELECT DATABASEPROPERTYEX (DB_NAME(DB_ID()), 'Edition') AS [Database Edition],
	   DATABASEPROPERTYEX (DB_NAME(DB_ID()), 'ServiceObjective') AS [Service Objective],
	   DATABASEPROPERTYEX (DB_NAME(DB_ID()), 'MaxSizeInBytes') AS [Max Size In Bytes],
	   DATABASEPROPERTYEX (DB_NAME(DB_ID()), 'IsXTPSupported') AS [Is XTP Supported]
	   OPTION (RECOMPILE);   

(1 row affected)

Total execution time: 00:00:00.119

Database Edition,Service Objective,Max Size In Bytes,Is XTP Supported
Standard,S0,268435456000,0


# Appendix

**These six Pluralsight Courses go into more detail about how to run these queries and interpret the results**

- [Azure SQL Database: Diagnosing Performance Issues with DMVs](https://bit.ly/2meDRCN)
- [SQL Server 2017: Diagnosing Performance Issues with DMVs](https://bit.ly/2FqCeti)
- [SQL Server 2017: Diagnosing Configuration Issues with DMVs](https://bit.ly/2MSUDUL)
- [SQL Server 2014 DMV Diagnostic Queries – Part 1](https://bit.ly/2plxCer) 
- [SQL Server 2014 DMV Diagnostic Queries – Part 2](https://bit.ly/2IuJpzI)
- [SQL Server 2014 DMV Diagnostic Queries – Part 3](https://bit.ly/2FIlCPb)




[Microsoft Visual Studio Dev Essentials](https://bit.ly/2qjNRxi)

[Microsoft Azure Learn](https://bit.ly/2O0Hacc)